In [18]:
!pip install transformers sentence-transformers scikit-learn pandas numpy
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install --upgrade jupyter ipywidgets



[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cpu



[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import util
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
df = pd.read_csv("../../data/Database_Cleaned.csv")

In [21]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1168.99it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [22]:
df["combined_features"] = (
        df["Title"] + ". " +
        df["Overview"] + ". Genres: " +
        df["Genre"]
)

In [23]:
corpus_embeddings = model.encode(
    df["combined_features"].tolist(),
    convert_to_numpy=True,
    show_progress_bar=True
)

Batches: 100%|██████████| 308/308 [00:56<00:00,  5.41it/s]


In [24]:
user_input = """
I want a romantic movies with a strong female lead set in a historical period.
"""
user_embedding = model.encode(
    [user_input],
    convert_to_numpy=True
)

In [25]:
similarities = cosine_similarity(
    user_embedding,
    corpus_embeddings
)[0]

In [26]:
df["semantic_score"] = similarities

top_movies = df.sort_values(
    by="semantic_score",
    ascending=False
).head(5)

top_movies[["Title", "Genre", "semantic_score"]]

,Title,Genre,semantic_score
2597,Date Movie,"Comedy, Romance",0.545401
1961,Flashback,Comedy,0.529786
2806,The World to Come,"Romance, Drama, Western",0.522475
1699,Little Women,"Drama, Romance",0.516619
6688,Killing Me Softly,Thriller,0.509713
